In [11]:
from datetime import datetime

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

from IPython.core.display import HTML                                   #Função renderização html no Jupyter
display(HTML("<style>pre { white-space: pre !important; }</style>"))    #Função renderização html no Jupyter

In [12]:
spark = SparkSession.builder.appName("MeuApp").getOrCreate()

In [13]:
TODAY = datetime.now().strftime("%Y-%m-%d")

In [20]:
def uppercase_columns(df):
    for col in df.columns:
        df = df.withColumnRenamed(col, col.upper())
    return df

def reg_news(df_new, df_old):

    df_reg = df_new.exceptAll(df_old)

    return df_reg

    return df_reg

In [15]:
df_csv = spark.read.option("delimiter", ",").option("header", True).csv("files/data_parameter.csv")
df_csv = uppercase_columns(df_csv)

df_csv = df_csv\
.withColumn("NAME_PARAM", F.upper(F.col("NAME_PARAM")))\
.withColumn("PARAM_TYPE", F.col("PARAM_TYPE").cast("integer"))\
.withColumn("PARAM_TARGET", F.col("PARAM_TARGET").cast("integer"))\
.withColumn("PARAM_STDEV", F.col("PARAM_STDEV").cast("float"))\
.withColumn("DATE_REF", F.to_date(F.lit(f"{TODAY}"),"yyyy-MM-dd").cast("date"))\

df_csv.show(truncate=False)

+---+----------+----------+------------+-----------+----------+
|ID |NAME_PARAM|PARAM_TYPE|PARAM_TARGET|PARAM_STDEV|DATE_REF  |
+---+----------+----------+------------+-----------+----------+
|1  |ALPHA     |101       |90          |1.0        |2024-06-12|
|2  |BRAVO     |201       |80          |1.0        |2024-06-12|
|3  |CHARLIE   |301       |90          |1.0        |2024-06-12|
|4  |DELTA     |401       |80          |1.0        |2024-06-12|
|5  |ECHO      |501       |90          |1.0        |2024-06-12|
|6  |FHOX      |601       |90          |1.0        |2024-06-12|
+---+----------+----------+------------+-----------+----------+



In [16]:
df_base = spark.read.parquet("spark-warehouse/parameters")
df_base.show(truncate=False)

+---+----------+----------+------------+-----------+----------+
|ID |NAME_PARAM|PARAM_TYPE|PARAM_TARGET|PARAM_STDEV|DATE_REF  |
+---+----------+----------+------------+-----------+----------+
|1  |ALPHA     |101       |90          |1.0        |2024-06-12|
|2  |BRAVO     |201       |80          |1.0        |2024-06-12|
|3  |CHARLIE   |301       |90          |1.0        |2024-06-12|
+---+----------+----------+------------+-----------+----------+



In [17]:
df_new = df_csv
df_new = df_new.drop("DATE_REF")
df_old = df_base
df_old = df_old.drop("DATE_REF")

In [18]:
df_new.show(truncate=False)
df_old.show(truncate=False)

+---+----------+----------+------------+-----------+
|ID |NAME_PARAM|PARAM_TYPE|PARAM_TARGET|PARAM_STDEV|
+---+----------+----------+------------+-----------+
|1  |ALPHA     |101       |90          |1.0        |
|2  |BRAVO     |201       |80          |1.0        |
|3  |CHARLIE   |301       |90          |1.0        |
|4  |DELTA     |401       |80          |1.0        |
|5  |ECHO      |501       |90          |1.0        |
|6  |FHOX      |601       |90          |1.0        |
+---+----------+----------+------------+-----------+

+---+----------+----------+------------+-----------+
|ID |NAME_PARAM|PARAM_TYPE|PARAM_TARGET|PARAM_STDEV|
+---+----------+----------+------------+-----------+
|1  |ALPHA     |101       |90          |1.0        |
|2  |BRAVO     |201       |80          |1.0        |
|3  |CHARLIE   |301       |90          |1.0        |
+---+----------+----------+------------+-----------+



In [21]:
df = reg_news(df_new, df_old)
df.show(truncate=False)

+---+----------+----------+------------+-----------+
|ID |NAME_PARAM|PARAM_TYPE|PARAM_TARGET|PARAM_STDEV|
+---+----------+----------+------------+-----------+
|4  |DELTA     |401       |80          |1.0        |
|5  |ECHO      |501       |90          |1.0        |
|6  |FHOX      |601       |90          |1.0        |
+---+----------+----------+------------+-----------+



In [ ]:
# df_params.write.format("parquet")\
#     .partitionBy("DATE_REF")\
#     .mode("overwrite")\
#     .saveAsTable("parameters")